In [1]:
!pip install tensorflow

  Using cached tensorflow-2.16.1-cp311-cp311-win_amd64.whl.metadata (3.5 kB)
Using cached tensorflow-2.16.1-cp311-cp311-win_amd64.whl (2.1 kB)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from keras.models import load_model 

# Step 1: Read the CSV file and extract the 'Rate' column
def load_data(file_path):
    df = pd.read_csv(file_path)
    series = df['Rate'].values  # Assuming the column is named 'Rate'
    return series

# Step 2: Normalize the column values
def normalize_data(series):
    scaler = MinMaxScaler(feature_range=(0, 1))
    series = series.reshape(-1, 1)
    scaled_series = scaler.fit_transform(series)
    return scaled_series, scaler

# Step 3: Create the training data
def create_dataset(series, time_step=100):
    X, y = [], []
    for i in range(len(series) - time_step):
        X.append(series[i:(i + time_step), 0])
        y.append(series[i + time_step, 0])
    return np.array(X), np.array(y)

# Step 4: Build and train the LSTM model
def build_and_train_model(X_train, y_train, epochs=50, batch_size=100):
    model = Sequential()
    model.add(LSTM(units=50, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(LSTM(units=50, activation='relu', return_sequences=False))
    model.add(Dense(units=1))
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
    # Save the model
    model.save("predict.h5") 
    return model

# Step 5: Predict the next 10 values
def predict_future_values(model, series, scaler, time_step=100, n_predictions=10):
    predictions = []
    input_data = series[-time_step:].reshape(1, time_step, 1)
    
    for _ in range(n_predictions):
        pred = model.predict(input_data)
        predictions.append(pred[0, 0])
        
        # Update input_data with the new prediction
        pred = np.reshape(pred, (1, 1, 1))  # Reshape pred to match input_data's shape
        input_data = np.append(input_data[:, 1:, :], pred, axis=1)
    
    # Inverse transform predictions to original scale
    predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))
    return predictions

# Putting it all together
file_path = 'Rate.csv'
series = load_data(file_path)
scaled_series, scaler = normalize_data(series)
X_train, y_train = create_dataset(scaled_series)

# Reshape X_train to be [samples, time steps, features]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

model = build_and_train_model(X_train, y_train)

predictions = predict_future_values(model, scaled_series, scaler)
print(predictions)

Epoch 1/50


C:\Users\Saket Kumar\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


42/42 ━━━━━━━━━━━━━━━━━━━━ 5s 63ms/step - loss: 0.1736
Epoch 2/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 62ms/step - loss: 0.0025
Epoch 3/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 62ms/step - loss: 4.1208e-04
Epoch 4/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 64ms/step - loss: 3.1878e-04
Epoch 5/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 64ms/step - loss: 3.1000e-04
Epoch 6/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - loss: 2.8837e-04
Epoch 7/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 64ms/step - loss: 2.6228e-04
Epoch 8/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step - loss: 2.5054e-04
Epoch 9/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 71ms/step - loss: 2.5624e-04
Epoch 10/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step - loss: 2.1363e-04
Epoch 11/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 65ms/step - loss: 2.5079e-04
Epoch 12/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 77ms/step - loss: 2.2635e-04
Epoch 13/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - loss: 2.1687e-04
Epoch 14/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step - loss: 1.9104e-04
Epoch 15/50
42/42 ━━━━━━━━━━━━━━━━